This notebook uses SimpleITK to coregister image stacks by first centering the images with an affine registration (linear transform) and then performs B-spline registration (non-linear) to optimize the registration.

To install SimpleITK, from the command line prompt, execute:
mamba install -c conda-forge simpleitk

if you cannot launch jupyter notebook in the terminal, try:
python -m jupyterlab

For information about installation and background on the code look here: 

https://simpleitk.readthedocs.io/en/master/gettingStarted.html

https://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/


In [21]:
# First we check if all necessary packages are installed correctly in the active environment. 

from __future__ import print_function
import importlib
from distutils.version import LooseVersion

# check that all packages are installed (see requirements.txt file)
required_packages = {'jupyter', 
                     'numpy',
                     'ipywidgets',
                     'scipy',
                     'pandas',
                     'numba',
                     'multiprocess',
                     'SimpleITK',
                     'readlif',
                     'skimage'
                    }

problem_packages = list()
# Iterate over the required packages: If the package is not installed
# ignore the exception. 
for package in required_packages:
    try:
        p = importlib.import_module(package)        
    except ImportError:
        problem_packages.append(package)
    
if len(problem_packages) == 0:
    print('All is well.')
else:
    print('The following packages are required but not installed: ' \
          + ', '.join(problem_packages))

The following packages are required but not installed: multiprocess, numba


In [22]:
# load the packages
from readlif.reader import LifFile
import SimpleITK as sitk
import numpy as np
import time
from skimage import io
import os

print(sitk.Version())

SimpleITK Version: 2.4.0 (ITK 5.4)
Compiled: Aug 13 2024 14:20:50



Load the lif files for coregistration

In [48]:
# Define your reference (day1) LIF file and the folder with other days
day1_lif_path = r"D:\Thijs\image_processing-feature-czi-output\Ravi_testing\day1\HIDE_FINAL_TMA002_day1_R2.lif"
other_days_folder = r"D:\Thijs\image_processing-feature-czi-output\Ravi_testing\otherdays"  # Folder with day2, day3, etc.
output_path = r"D:\Thijs\image_processing-feature-czi-output\Ravi_testing\HIDE_FINAL_TMA002_all_days_R2_aligned.tif"

In [49]:
# Helper function to extract channels from a LIF file
def extract_channels(lif_file, z=0, t=0, c=4):
    """
    Extracts channels from a LIF file as NumPy arrays.

    Args:
        lif_file (LifFile): The LIF file object.
        z (int): The z-index (depth), assuming 2D images, set it to 0.
        t (int): The t-index (time), assuming no time-lapse, set it to 0.
        c (int): Number of channels. Defaults to 4.

    Returns:
        list: A list of NumPy arrays, one for each channel.
    """
    img = lif_file.get_image(0)  # Assuming there's only 1 image per LIF file
    num_channels = c  # Set number of channels (4 in your case)
    channels = []

    for c in range(num_channels):
        frame = img.get_frame(z=z, t=t, c=c)
        channel = np.uint16(np.array(frame))
        channels.append(channel)

    return channels

# Function to convert channels to SimpleITK format
def convert_to_sitk(channel_arrays):
    sitk_images = []
    for channel_data in channel_arrays:
        sitk_image = sitk.GetImageFromArray(channel_data)
        sitk_image = sitk.Cast(sitk_image, sitk.sitkFloat32)
        sitk_images.append(sitk_image)
    return sitk_images

In [50]:
# Function to apply affine + BSpline transformations with progress tracking
def apply_affine_bspline_transformation(fixed_image, moving_channels, final_affine_transform):
    print("\nStarting affine transformation for all channels...")
    affine_start_time = time.time()
    
    # Apply affine transformation to each channel
    affine_transformed_channels = []
    for idx, moving_channel in enumerate(moving_channels):
        print(f"Applying affine transformation to channel {idx + 1}/{len(moving_channels)}...")
        affine_transformed_channel = sitk.Resample(
            moving_channel, fixed_image, final_affine_transform, sitk.sitkLinear, 0.0, moving_channel.GetPixelID()
        )
        affine_transformed_channels.append(affine_transformed_channel)
    
    affine_end_time = time.time()
    print(f"Affine transformation completed for all channels. Time taken: {affine_end_time - affine_start_time:.2f} seconds.")
    
    print("\nStarting BSpline transformation for all channels...")
    bspline_start_time = time.time()
    
    # Initialize BSpline transform
    bspline_mesh_size = [10] * fixed_image.GetDimension()  # Adjust mesh size as needed, increase value for larger images
    initial_bspline_transform = sitk.BSplineTransformInitializer(fixed_image, bspline_mesh_size)

    # Create registration method for B-spline
    bspline_registration_method = sitk.ImageRegistrationMethod()
    bspline_registration_method.SetMetricAsMattesMutualInformation()
    bspline_registration_method.SetOptimizerAsGradientDescentLineSearch(
        learningRate=10.0, numberOfIterations=25, convergenceMinimumValue=1e-6, convergenceWindowSize=10
    )
    bspline_registration_method.SetInterpolator(sitk.sitkLinear)
    bspline_registration_method.SetInitialTransformAsBSpline(
        initial_bspline_transform, inPlace=True, scaleFactors=[1, 3, 6]
    )
    bspline_registration_method.SetShrinkFactorsPerLevel([32, 16, 8, 4])
    bspline_registration_method.SetSmoothingSigmasPerLevel([32, 16, 8, 4])
    bspline_registration_method.SetMetricSamplingStrategy(bspline_registration_method.RANDOM)
    bspline_registration_method.SetMetricSamplingPercentage(0.1)

    # Perform B-spline registration on the affine-transformed DAPI channel (first channel)
    print("Starting BSpline registration on the DAPI channel (channel 1)...")
    bspline_registration_start_time = time.time()
    final_bspline_transform = bspline_registration_method.Execute(fixed_image, affine_transformed_channels[0])
    bspline_registration_end_time = time.time()
    print(f"BSpline registration for DAPI channel completed. Time taken: {bspline_registration_end_time - bspline_registration_start_time:.2f} seconds.")

    # Apply BSpline transformation to all channels
    bspline_transformed_channels = []
    for idx, affine_channel in enumerate(affine_transformed_channels):
        print(f"Applying BSpline transformation to channel {idx + 1}/{len(affine_transformed_channels)}...")
        bspline_transformed_channel = sitk.Resample(
            affine_channel, fixed_image, final_bspline_transform, sitk.sitkLinear, 0.0, affine_channel.GetPixelID()
        )
        bspline_transformed_channels.append(bspline_transformed_channel)
    
    bspline_end_time = time.time()
    print(f"BSpline transformation completed for all channels. Time taken: {bspline_end_time - bspline_start_time:.2f} seconds.")
    print(f"Total time for affine + BSpline transformation: {bspline_end_time - affine_start_time:.2f} seconds.")

    return bspline_transformed_channels

# Start timing the entire process
start_time = time.time()

# Load day 1 LIF file and extract channels
lif_day1 = LifFile(day1_lif_path)
channels_day1 = extract_channels(lif_day1)
sitk_images_day1 = convert_to_sitk(channels_day1)

# Reference image (DAPI channel from day1)
fixed_image = sitk_images_day1[0]

# Initialize an empty list to store all channels from all days
all_channels = []

# Add day1 channels to the list
all_channels.extend(channels_day1)

# Get the total number of files for progress tracking
total_files = len([f for f in os.listdir(other_days_folder) if f.endswith(".lif") and "day1" not in f])

# Iterate over all LIF files in the other_days_folder in alphabetical order
for i, filename in enumerate(sorted(os.listdir(other_days_folder))):
    if filename.endswith(".lif") and "day1" not in filename:
        file_start_time = time.time()
        print(f"\nProcessing file {i + 1}/{total_files}: {filename}")

        lif_file_path = os.path.join(other_days_folder, filename)
        lif_file = LifFile(lif_file_path)
        channels_current_day = extract_channels(lif_file)
        sitk_images_current_day = convert_to_sitk(channels_current_day)

        # Initialize and run the affine registration
        registration_method = sitk.ImageRegistrationMethod()
        registration_method.SetMetricAsMattesMutualInformation()
        registration_method.SetOptimizerAsGradientDescent(learningRate=1, numberOfIterations=1000, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
        registration_method.SetInterpolator(sitk.sitkLinear)

        # Set the initial transform for affine registration
        initial_transform = sitk.CenteredTransformInitializer(fixed_image, sitk_images_current_day[0], sitk.AffineTransform(fixed_image.GetDimension()))
        registration_method.SetInitialTransform(initial_transform)
        registration_method.SetOptimizerScalesFromPhysicalShift()
        registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
        registration_method.SetMetricSamplingPercentage(0.1)

        # Perform affine registration
        final_affine_transform = registration_method.Execute(fixed_image, sitk_images_current_day[0])

        # Apply affine + BSpline transformations
        aligned_images = apply_affine_bspline_transformation(fixed_image, sitk_images_current_day, final_affine_transform)

        # Convert aligned channels to NumPy arrays and ensure they are 16-bit
        aligned_arrays = [sitk.GetArrayFromImage(aligned_image).astype(np.uint16) for aligned_image in aligned_images]

        # Append aligned channels to the all_channels list
        all_channels.extend(aligned_arrays)

        # Track file processing time
        file_end_time = time.time()
        print(f"Finished processing {filename}. Time taken: {file_end_time - file_start_time:.2f} seconds")



Processing file 1/3: HIDE_FINAL_TMA002_day2_R2.lif

Starting affine transformation for all channels...
Applying affine transformation to channel 1/4...
Applying affine transformation to channel 2/4...
Applying affine transformation to channel 3/4...
Applying affine transformation to channel 4/4...
Affine transformation completed for all channels. Time taken: 2.59 seconds.

Starting BSpline transformation for all channels...
Starting BSpline registration on the DAPI channel (channel 1)...
BSpline registration for DAPI channel completed. Time taken: 3102.00 seconds.
Applying BSpline transformation to channel 1/4...
Applying BSpline transformation to channel 2/4...
Applying BSpline transformation to channel 3/4...
Applying BSpline transformation to channel 4/4...
BSpline transformation completed for all channels. Time taken: 3143.36 seconds.
Total time for affine + BSpline transformation: 3145.96 seconds.
Finished processing HIDE_FINAL_TMA002_day2_R2.lif. Time taken: 4821.62 seconds

Pro

In [51]:
# Stack all aligned channels into a single multichannel image
combined_image = np.stack(all_channels, axis=0)

In [52]:
# Save the final combined image as a multi-channel TIFF
io.imsave(output_path, combined_image.astype('uint16'), check_contrast=False, imagej=True)

print(f"Aligned multichannel TIFF saved to {output_path}")

d:\Thijs\image_processing-feature-czi-output\.conda\lib\site-packages\tifffile\tifffile.py:3794: UserWarning: <tifffile.TiffWriter 'HIDE_FINAL_TMA0…s_R2_aligned.tif'> truncating ImageJ file
  warnings.warn(


Aligned multichannel TIFF saved to D:\Thijs\image_processing-feature-czi-output\Ravi_testing\HIDE_FINAL_TMA002_all_days_R2_aligned.tif


In [53]:
# Total time for the entire process
end_time = time.time()
print(f"Total time for processing all files: {end_time - start_time:.2f} seconds")

Total time for processing all files: 14669.67 seconds
